In [ ]:
import os
import pandas

dir_path = './Data_RNN'

df = pandas.DataFrame()
    
for file_name in os.listdir(dir_path):
    file_path = os.path.join(dir_path, file_name)
    if not os.path.isdir(file_path):
        ext = os.path.splitext(file_path)[-1]
        if ext == '.txt': 
            print(file_path)
            
            curr_df = pandas.read_csv(file_path, delim_whitespace=True, header=None, usecols=[6])
            df = df.append(curr_df)

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

dataset = df.values
dataset = dataset.astype('float32')
dataset = np.log10(dataset)

# normalize
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(dataset)
dataset = np.reshape(dataset, (11, 288, 1))

#dataset.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

for ds in dataset:
    plt.plot(ds)
    
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

model = Sequential()
model.add(LSTM(32, batch_input_shape=(1, 20, 1), stateful=True, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32, batch_input_shape=(1, 20, 1), stateful=True))
model.add(Dropout(0.3))
model.add(Dense(1))

#model = Sequential()
#model.add(LSTM(100, batch_input_shape=(1, 20, 1), stateful=True))
#model.add(Dropout(0.3))
#model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        dataX.append(dataset[i:( i + look_back), 0])
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
for epoch in range(5):
    
    for event_idx in range(10):
    
        print('epoch : ' + str(epoch) + ' | event id : ' + str(event_idx))
        
        trainX, trainY = create_dataset(dataset[event_idx], 20)
        trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1)) # samples, time steps, features

        model.fit(trainX, trainY, nb_epoch=1, batch_size=1, verbose=2, shuffle=False)
        model.reset_states()

In [ ]:
testX, testY = create_dataset(dataset[10], 20)
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

testPredict = [testX[0]]

In [ ]:
look_ahead = 288 - 20
predictions = np.zeros((look_ahead, 1))
    
for i in range(look_ahead):
    prediction = model.predict(np.array([testPredict[-1]]), batch_size=1, verbose=0)
    predictions[i] = prediction
    testPredict.append(np.vstack([testPredict[-1][1:], prediction]))
    
model.reset_states()

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(np.arange(look_ahead), predictions,'r', label="prediction")
plt.plot(np.arange(look_ahead), testY, label="test function")
plt.legend()
plt.show()

In [ ]:
testX, testY = create_dataset(dataset[10], 20)
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

testPredict = [testX[0]]

look_real = 100
look_ahead = 287-100

predictions = np.zeros((look_real + look_ahead, 1))

for i in range(look_real):
    
    prediction = model.predict(np.array([testPredict[-1]]), batch_size=1, verbose=0)
    predictions[i] = prediction
    testPredict.append(np.vstack([testPredict[-1][1:], testY[i]]))
    
for i in range(look_ahead):
    prediction = model.predict(np.array([testPredict[-1]]), batch_size=1, verbose=0)
    predictions[look_real + i] = prediction
    testPredict.append(np.vstack([testPredict[-1][1:], prediction]))
    
model.reset_states()  

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(np.arange(287-20), predictions,'r', label="prediction")
plt.plot(np.arange(287-20), testY, label="test function")
plt.legend()
plt.show()